<!--
Doc Writer email@nixdabei.de
v0.0.1, 2022-02-19
-->
[Home](../index.ipynb) /  Leistungstreiber L9110: Simple car
***
# Leistungstreiber L9110: Simple Car

## Klasse: Motor

In [91]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine

class Motor :
    def __init__( self, 
            iPinA, 
            iPinB,
            iFrequency     = 1000,
            iDutyMinLeft   =    0,  # Duty for speed = 0:              motor turns left
            iDutyMaxLeft   = 1023,  # Duty for speed = fSpeedMaxLeft:  motor turns left
            iDutyMinRight  =    0,  # Duty for speed = 0:              motor turns right
            iDutyMaxRight  = 1023,  # Duty for speed = fSpeedMaxRight: motor turns right
            fSpeedMaxLeft  = -1.0,  # Minimal value for setSpeed() --> iDutyMaxLeft
            fSpeedMaxRight =  1.0   # Maximal value for setSpeed() --> iDutyMaxRight
        ) :
        self._iFrequency    = iFrequency
        self._fScaleLeft    = (iDutyMaxLeft  - iDutyMinLeft )/fSpeedMaxLeft
        self._fScaleRight   = (iDutyMaxRight - iDutyMinRight)/fSpeedMaxRight
        self._iDutyMinLeft  = iDutyMinLeft
        self._iDutyMaxLeft  = iDutyMaxLeft
        self._iDutyMinRight = iDutyMinRight
        self._iDutyMaxRight = iDutyMaxRight
        
        self._fCurSpeed     = 0
        
        self._pwmA = machine.PWM( machine.Pin(iPinA, machine.Pin.OUT), freq=self._iFrequency, duty=0)
        self._pwmB = machine.PWM( machine.Pin(iPinB, machine.Pin.OUT), freq=self._iFrequency, duty=0)
        
        self._pwmDir   = self._pwmA
        self._pwmSpeed = self._pwmB
        
    def speedToDuty( self, fSpeed ) :
        if fSpeed == 0 :
            return 0
        elif fSpeed < 0 :
            return min( self._iDutyMaxLeft, max( self._iDutyMinLeft, int(self._iDutyMinLeft  + fSpeed*self._fScaleLeft +0.5) ) )
        else :
            return min( self._iDutyMaxRight, max( self._iDutyMinRight, int(self._iDutyMinRight + fSpeed*self._fScaleRight+0.5) ) )
            
    
    def setLeft( self ) :
        self._pwmDir   = self._pwmA
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmB
      
    def setRight( self ) :
        self._pwmDir   = self._pwmB
        self._pwmDir.duty( 0 )
        self._pwmSpeed = self._pwmA
      
    def setSpeed( self, fSpeed ) :
        if fSpeed != self._fCurSpeed :
            iDuty = self.speedToDuty( fSpeed )
            
            if fSpeed > 0 and self._fCurSpeed <= 0 :
                self.setLeft()

            elif fSpeed < 0 and self._fCurSpeed >= 0 :
                self.setRight()
            
            self._pwmSpeed.duty( iDuty )
        
        self._fCurSpeed = fSpeed
        
    def deinit( self ):
        self.setSpeed( 0 )

        self._pwmSpeed.deinit()
        self._pwmDir.deinit()        

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


## Simple Car

In [92]:
from time import sleep_ms

DELAY_MILLIS = 20

I_STEPS     = 100
F_SPEED_MIN = -1.0
F_SPEED_MAX =  1.0


motorL = Motor( 12, 13,
    iFrequency     = 300,
    iDutyMinLeft   = 330,
    iDutyMaxLeft   = 1023,
    iDutyMinRight  = 330,
    iDutyMaxRight  = 1023,
    fSpeedMaxLeft  = -1.0,
    fSpeedMaxRight =  1.0 )

motorR = Motor( 18, 22,
    iFrequency     = 300,
    iDutyMinLeft   = 0,
    iDutyMaxLeft   = 1023,
    iDutyMinRight  = 0,
    iDutyMaxRight  = 1023,
    fSpeedMaxLeft  = -1.0,
    fSpeedMaxRight =  1.0 )


fSpeed    = F_SPEED_MIN
fStep     = (F_SPEED_MAX - F_SPEED_MIN)/I_STEPS


try:    
    for i in range( 2 ):
        while fSpeed <= F_SPEED_MAX:
            #motorL.setSpeed(  fSpeed )
            motorR.setSpeed( -fSpeed )
            sleep_ms(DELAY_MILLIS)
            fSpeed += fStep
        sleep_ms( 1000 )
        while fSpeed >= F_SPEED_MIN:
            motorL.setSpeed(  fSpeed )
            #motorR.setSpeed( -fSpeed )
            sleep_ms(DELAY_MILLIS)
            fSpeed-=fStep
            
except KeyboardInterrupt: pass
        
motorL.deinit()
motorR.deinit()
    
print( "Done" )        

..

*** Sending Ctrl-C

Done


In [ ]:
#%serialconnect --port=COM3 --baud=115200 # für Windows
%serialconnect # für Linux

from machine import Pin, PWM
import time

pin = PWM( Pin(22, Pin.OUT), freq=50, duty=0) # 25 HelTec, 13 Croduino, NodeMCU: extern 14

N = 5

for i in range( N ):
    for iDuty in range( 0, 10000, 2 ):
        pin.duty(iDuty)
        time.sleep( 0.001 )

    for iDuty in range( 1001, 0, -2 ):
        pin.duty(iDuty)
        time.sleep( 0.001 )
        
pin.deinit() # falls der Pin im selbem Porgramm noch mal als on/off- Pin verwendet werden soll.
    
print( "\nDone." )